# 5 - Creador PPTX


<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/rag_2.webp" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---OpenAI-API-KEY" data-toc-modified-id="1---OpenAI-API-KEY-1">1 - OpenAI API KEY</a></span></li><li><span><a href="#2---Probando-GPT4-desde-LangChain" data-toc-modified-id="2---Probando-GPT4-desde-LangChain-2">2 - Probando GPT4 desde LangChain</a></span></li><li><span><a href="#3---Cargando-archivo-PDF" data-toc-modified-id="3---Cargando-archivo-PDF-3">3 - Cargando archivo PDF</a></span></li><li><span><a href="#4---Modelo-de-embedding-y-base-de-datos-vectorial" data-toc-modified-id="4---Modelo-de-embedding-y-base-de-datos-vectorial-4">4 - Modelo de embedding y base de datos vectorial</a></span></li><li><span><a href="#5---Plantilla-de-prompt" data-toc-modified-id="5---Plantilla-de-prompt-5">5 - Plantilla de prompt</a></span></li><li><span><a href="#6---Creando-la-primera-cadena" data-toc-modified-id="6---Creando-la-primera-cadena-6">6 - Creando la primera cadena</a></span></li><li><span><a href="#7---Creando-la-segunda-cadena" data-toc-modified-id="7---Creando-la-segunda-cadena-7">7 - Creando la segunda cadena</a></span></li><li><span><a href="#8---Creando-el-archivo-pptx" data-toc-modified-id="8---Creando-el-archivo-pptx-8">8 - Creando el archivo pptx</a></span></li></ul></div>

## 1 - OpenAI API KEY

Para llevar a cabo este proyecto, necesitaremos una API KEY de OpenAI para utilizar el modelo GPT-4 Turbo. Esta API KEY se puede obtener en https://platform.openai.com/api-keys. Solo se muestra una vez, por lo que debe guardarse en el momento en que se obtiene. Por supuesto, necesitaremos crear una cuenta para obtenerla.

Guardamos la API KEY en un archivo `.env` para cargarla con la biblioteca dotenv y usarla como una variable de entorno. Este archivo se agrega a `.gitignore` para asegurarnos de que no pueda verse si subimos el código a GitHub, por ejemplo.


In [1]:
# importamos la API KEY

import os                           # libreria del sistema operativo
from dotenv import load_dotenv      # carga variables de entorno 


load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## 2 - Probando GPT4 desde LangChain

Vamos a probar la conexión de LangChain con el modelo GPT-4. Preguntemos qué es el canal de Suez.

In [2]:
from langchain_openai.chat_models import ChatOpenAI   

modelo = ChatOpenAI(model='gpt-4-turbo')

respuesta = modelo.invoke('What is the Suez Canal?')

respuesta.content

"The Suez Canal is a man-made waterway located in Egypt, connecting the Mediterranean Sea to the Red Sea. This canal is a critical shortcut for international maritime trade, allowing ships to pass between Europe and Asia without navigating around Africa via the Cape of Good Hope. \n\nThe canal was opened in 1869, after ten years of construction led by the French engineer Ferdinand de Lesseps under the auspices of the Egyptian government. It is one of the world's most heavily used shipping lanes and plays a crucial role in the economies of Egypt and numerous other countries, as it facilitates the fast and economically viable transportation of oil and other goods.\n\nThe canal has been expanded and modernized several times since its opening to accommodate larger ships and increased traffic. It remains a vital artery for global trade, making it strategically and economically significant on a global scale."

## 3 - Cargando archivo PDF 

Ahora, cargamos el archivo PDF [Amenazas recientes en el Mar Rojo](https://www.europarl.europa.eu/RegData/etudes/BRIE/2024/760390/EPRS_BRI(2024)760390_EN.pdf) previamente descargado, un informe publicado por el Parlamento Europeo en 2024.


In [3]:
os.listdir('../pdfs')

['2024_BRIEFING.pdf']

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [5]:
# cargar archivo pdf 

loader = PyPDFDirectoryLoader('../pdfs/')

paginas = loader.load()

In [6]:
# nº de paginas en el pdf

len(paginas)

8

## 4 - Modelo de embedding y base de datos vectorial


Los embeddings transforman datos, especialmente datos textuales, en un formato, generalmente un vector de números, que los algoritmos de aprendizaje automático pueden procesar de manera efectiva. Estos embeddings capturan las relaciones contextuales y los significados semánticos de palabras, frases o documentos, permitiendo diversas aplicaciones en IA.

Chroma DB es una base de datos de vectores de código abierto diseñada para almacenar y recuperar embeddings de vectores de manera eficiente. Es particularmente útil para mejorar los LLMs al proporcionar contexto relevante a las consultas de los usuarios. Chroma DB permite almacenar embeddings junto con metadatos, que luego pueden ser utilizados por LLMs o por motores de búsqueda semántica sobre datos textuales.


In [7]:
# modelo embedding de OpenAI

from langchain_openai.embeddings import OpenAIEmbeddings

vectorizador = OpenAIEmbeddings()

In [8]:
# guardando vectores en ChromaDB

from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(paginas, vectorizador, persist_directory='../chroma_db')

In [9]:
# objecto para recuperar 2 paginasdesde la base de datos

recuperador = chroma_db.as_retriever(search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25})

## 5 - Plantilla de prompt 

Los prompt templates son recetas predefinidas para generar instrucciones para modelos de lenguaje.

Un template puede incluir instrucciones, contexto y preguntas específicas adecuadas para una tarea determinada. LangChain proporciona herramientas para crear y trabajar con templates de instrucciones y también busca crear templates independientes del modelo para facilitar la reutilización de templates existentes en diferentes modelos de lenguaje.

Escribimos el template para el primer LLM para crear la lista de puntos clave.

In [10]:
from langchain.prompts import ChatPromptTemplate

In [11]:
template = '''
            Given the context below and the question, 
            please generate a header and 10 bullet points.
            List with numbers the bullet points.
            Summarize each bullet point in 40 words.
            
            Put a line separator after `:` symbol.

            Context: {context}

            Question: {question}
            '''


prompt = ChatPromptTemplate.from_template(template)

## 6 - Creando la primera cadena

La primera cadena crea la lista de puntos clave.

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
consulta = 'What are the endnotes of the briefing?'

In [15]:
in_chain = {'context': recuperador, 'question': RunnablePassthrough()} | prompt | modelo | parser


respuesta = in_chain.invoke(consulta)

Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16


In [16]:
respuesta.split("\n")

['Header: **Endnotes of the 2024 Briefing on Recent Threats in the Red Sea**',
 '',
 '1. **Page Information**:',
 '   - Document is sourced from "../pdfs/2024_BRIEFING.pdf" and is located on page 0, with additional content on page 6.',
 '',
 '2. **Authorship and Graphics**:',
 '   - Authored by Angelos Delivorias with graphics by Samy Chahri.',
 '',
 '3. **Publishing Details**:',
 '   - Published by the European Parliamentary Research Service (EPRS), document number PE 760.390, dated March 2024.',
 '',
 '4. **Geopolitical Focus**:',
 '   - The briefing mainly discusses recent threats in the Red Sea, specifically targeting the economic impacts on both regional and EU levels.',
 '',
 '5. **Key Events and Responses**:',
 '   - Describes attacks by the Iran-backed Houthi militia on Western commercial ships near the Bab el-Mandeb Strait, leading to major shipping diversions.',
 '',
 '6. **Economic Consequences**:',
 '   - Highlights the economic repercussions, including increased shipping t

## 7 - Creando la segunda cadena

La segunda cadena escribe el código Python para crear el archivo pptx.

In [17]:
from langchain_openai import OpenAI

input_model = OpenAI(temperature=0, max_tokens=1024)

In [18]:
template = '''
            We have provided  information below.
            Given this information, please generate a python code with python-pptx for three 
            slide presentation with this information. 
            
            Put the title in the first slide, 
            5 bullet points in the second slide and another 5 bullet in the third slide.
            Put list number in each bullet point.
                        
            Separate the bullet points into separate texts with line separator.
            Set font size to 20 for each bullet point. 
            Save the file in ../pptx/Red Sea Security Threats.pptx path

            Information: {context}
            '''


prompt = ChatPromptTemplate.from_template(template)

In [19]:
out_chain = prompt | input_model | parser

In [20]:
output = out_chain.invoke({'context': respuesta})

In [21]:
output.split('\n')

['',
 '# Import necessary libraries',
 'from pptx import Presentation',
 'from pptx.util import Pt',
 '',
 '# Create new presentation',
 'prs = Presentation()',
 '',
 '# Add first slide with title',
 'slide = prs.slides.add_slide(prs.slide_layouts[0])',
 'title = slide.shapes.title',
 'title.text = "Endnotes of the 2024 Briefing on Recent Threats in the Red Sea"',
 '',
 '# Add second slide with 5 bullet points',
 'slide = prs.slides.add_slide(prs.slide_layouts[1])',
 'shapes = slide.shapes',
 'body_shape = shapes.placeholders[1]',
 'tf = body_shape.text_frame',
 '',
 '# Add bullet points with font size 20',
 'p = tf.add_paragraph()',
 'p.text = "1. Page Information:"',
 'p.font.size = Pt(20)',
 '',
 'p = tf.add_paragraph()',
 'p.text = "- Document is sourced from \\"../pdfs/2024_BRIEFING.pdf\\" and is located on page 0, with additional content on page 6."',
 'p.font.size = Pt(20)',
 '',
 'p = tf.add_paragraph()',
 'p.text = "2. Authorship and Graphics:"',
 'p.font.size = Pt(20)',
 '',


## 8 - Creando el archivo pptx 

Ahora tenemos el código Python y vamos a crear el archivo pptx mediante la función `exec()`. Esta función se usa para la ejecución dinámica de programas en Python, que pueden ser una cadena o un objeto de código. Si es una cadena, esta se analiza como un conjunto de declaraciones de Python que luego se ejecutan, a menos que ocurra un error de sintaxis. Con la cadena de salida del LLM, ejecutamos el código.


In [22]:
# ejemplo

exec('print(2+2)')

4


In [23]:
# ejecutando el codigo de python

exec(output)